In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from skimage.exposure import histogram
from skimage.color import rgb2hsv
from pathlib import Path
from urlpath import URL
import json
import glob 
import numpy as np
import seaborn as sns
from skimage.measure import shannon_entropy
import string
import nltk
nltk.download('words')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from ipywidgets import IntProgress


import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

# https://pypi.org/project/pytesseract/
# sudo apt install tesseract-ocr
import pytesseract
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# from utils import cleanOcrText
pd.options.display.max_columns = None



[nltk_data] Downloading package words to /home/lilatee/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lilatee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lilatee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv(f"../../../our_data/Etap5/Etap1/checkpoint1.csv")
df = df.loc[:, ["video_id", "thumbnail_link"]]
print(df.shape)
df[:3]

(8474, 2)


/home/lilatee/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,video_id,thumbnail_link
0,tOpTQ3Fuf9s,https://i.ytimg.com/vi/tOpTQ3Fuf9s/default.jpg
1,k0DqRstCgj4,https://i.ytimg.com/vi/k0DqRstCgj4/default.jpg
2,1CYH4F1KCw8,https://i.ytimg.com/vi/1CYH4F1KCw8/default.jpg


### Pobieranie thumbnaili

In [8]:
def readJSONList(path):
    with open(path, 'r') as file:
        loaded_list = json.loads(file.read())
    return loaded_list

def writeJSONList(path, list_to_save):
    with open(path, 'w') as file:
        file.write(json.dumps(list_to_save))

def downloadThumbnails(df, quality='hq'):
    list_of_ids_without_thumbnail = []
    ids = df["video_id"].values
    urls = df["thumbnail_link"].values
    
    Path("./../../../data/thumbnailsHQ/Etap5").mkdir(parents=True, exist_ok=True)
    savepath = Path("./../../../data/thumbnailsHQ/Etap5")
    filename = quality + 'default.jpg'
    
    for id, url in zip(ids, urls): 
        temp_url = URL(url)
        temp_url = temp_url.parent / filename
        
        try:
            img = io.imread(str(temp_url))
        except Exception:
            list_of_ids_without_thumbnail.append(id)
            print(f"{id} no thumbnail")
            continue
        io.imsave(savepath / (id + ".png") , img)
    
    writeJSONList(savepath / "ids_without_thumbnail.txt", list_of_ids_without_thumbnail)
        
    return list_of_ids_without_thumbnail

In [11]:
list_of_ids_without_thumbnail = downloadThumbnails(df)
# df.loc[df.video_id=="0Yp9WS6cJFA",:]

<ipython-input-8-7ef54d9d684d>:29: UserWarning: ../../../data/thumbnailsHQ/Etap5/_SWSdUPMLMI.png is a low contrast image
  io.imsave(savepath / (id + ".png") , img)
<ipython-input-8-7ef54d9d684d>:29: UserWarning: ../../../data/thumbnailsHQ/Etap5/oakC_u5vbdU.png is a low contrast image
  io.imsave(savepath / (id + ".png") , img)
<ipython-input-8-7ef54d9d684d>:29: UserWarning: ../../../data/thumbnailsHQ/Etap5/1BSlrD-HEfU.png is a low contrast image
  io.imsave(savepath / (id + ".png") , img)
<ipython-input-8-7ef54d9d684d>:29: UserWarning: ../../../data/thumbnailsHQ/Etap5/QaXD2oBJpFg.png is a low contrast image
  io.imsave(savepath / (id + ".png") , img)


BFKbrqjUXMI no thumbnail


In [13]:
import json
etap5 = open("./../../../data/thumbnailsHQ/Etap5/ids_without_thumbnail.txt", 'r')
etap5 = json.loads(etap5.read())
print(f"Etap5: {len(etap5)}")

Etap5: 1


### Średnia wartość kanałów w obrazie, odchylenie, histogramy, entropia, ocr tesseract

In [18]:
%%time
# Odkomentować zapis jeśli trzeba!!!
imgs_path = Path(f'./../../../data/thumbnailsHQ/Etap5')

for i, img_path in enumerate(glob.glob(f"{imgs_path}/*.png")): 
    path = Path(img_path)
    img = io.imread(path)
    
    list_of_pixels = img.reshape(img.shape[0]*img.shape[1], 3)
    mean_of_channels = np.mean(list_of_pixels, axis=0)
    std_of_channels = np.std(list_of_pixels, axis=0)

    df.loc[df.video_id == path.stem, "mean_r"] = mean_of_channels[0]
    df.loc[df.video_id == path.stem, "mean_g"] = mean_of_channels[1]
    df.loc[df.video_id == path.stem, "mean_b"] = mean_of_channels[2]
    
    df.loc[df.video_id == path.stem, "std_r"] = std_of_channels[0]
    df.loc[df.video_id == path.stem, "std_g"] = std_of_channels[1]
    df.loc[df.video_id == path.stem, "std_b"] = std_of_channels[2]
    
    hist_r, _ = histogram(img[:,:,0], 256)
    hist_g, _ = histogram(img[:,:,0], 256)
    hist_b, _ = histogram(img[:,:,0], 256)
    
    df.loc[df.video_id == path.stem, "hist_r"] = json.dumps(hist_r.tolist())
    df.loc[df.video_id == path.stem, "hist_g"] = json.dumps(hist_g.tolist())
    df.loc[df.video_id == path.stem, "hist_b"] = json.dumps(hist_b.tolist())
    
    img_hsv = rgb2hsv(img)

    list_of_pixels = img_hsv.reshape(img_hsv.shape[0]*img_hsv.shape[1], 3)
    mean_of_channels = np.mean(list_of_pixels, axis=0)
    std_of_channels = np.std(list_of_pixels, axis=0)

    df.loc[df.video_id == path.stem, "mean_h"] = mean_of_channels[0]
    df.loc[df.video_id == path.stem, "mean_s"] = mean_of_channels[1]
    df.loc[df.video_id == path.stem, "mean_v"] = mean_of_channels[2]
    
    df.loc[df.video_id == path.stem, "std_h"] = std_of_channels[0]
    df.loc[df.video_id == path.stem, "std_s"] = std_of_channels[1]
    df.loc[df.video_id == path.stem, "std_v"] = std_of_channels[2]
    
    
    df.loc[df.video_id == path.stem, "entropy"] = shannon_entropy(img)
    df.loc[df.video_id == path.stem, "thumb_ocr_text"] = pytesseract.image_to_string(img)
    
    if (i % 100 == 0):
        print(i, end='\r')

df["clean_thumb_ocr_text"] = df.apply(lambda row: cleanOcrText(row.thumb_ocr_text), axis=1)

df.loc[:, "thumb_ocr_text"] = df.loc[:, "thumb_ocr_text"].fillna('')
df.loc[:, "clean_thumb_ocr_text"] = df.loc[:, "clean_thumb_ocr_text"].fillna('')

df.to_csv(f"./../../../our_data/Etap5/Etap2/lilatee/01_tesseract.csv", index=False)
df[:3]

### CHECKPOINT

In [14]:
df = pd.read_csv(f"./../../../our_data/Etap5/Etap2/lilatee/01_tesseract.csv")
df[:5]

,video_id,thumbnail_link,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,mean_h,mean_s,mean_v,std_h,std_s,std_v,entropy,thumb_ocr_text,clean_thumb_ocr_text
0,tOpTQ3Fuf9s,https://i.ytimg.com/vi/tOpTQ3Fuf9s/default.jpg,89.439786,86.814786,91.361389,66.649746,64.668101,66.990815,"[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...",0.418176,0.139355,0.371523,0.377200,0.212690,0.270151,6.416138,\n,NaN
1,k0DqRstCgj4,https://i.ytimg.com/vi/k0DqRstCgj4/default.jpg,44.614514,21.644572,18.150214,59.224952,44.159844,37.943564,"[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...",0.263084,0.458768,0.175262,0.410572,0.402156,0.232071,4.978147,Se!\n\n \n\nSMTOWN~\nOFFICIAL\n\nI\n,official
2,1CYH4F1KCw8,https://i.ytimg.com/vi/1CYH4F1KCw8/default.jpg,89.025081,99.345885,116.328432,86.195281,90.707405,95.898890,"[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...",0.474761,0.326194,0.456745,0.252111,0.334823,0.375875,6.726162,\n,NaN
3,UdjEoGzXxmI,https://i.ytimg.com/vi/UdjEoGzXxmI/default.jpg,128.491042,125.048414,122.029821,85.459664,82.416423,79.346087,"[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...","[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...","[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...",0.209823,0.111204,0.513423,0.275839,0.187658,0.333337,6.146261,\n,NaN
4,cD8SYW8rjaQ,https://i.ytimg.com/vi/cD8SYW8rjaQ/default.jpg,57.089873,42.153490,36.812674,69.284572,58.369344,55.395476,"[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...","[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...","[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...",0.169770,0.377556,0.230208,0.240235,0.336576,0.273651,6.199159,\n,NaN


### Lepszy OCR

#### [Wykrywanie napisów - https://github.com/clovaai/deep-text-recognition-benchmark](https://github.com/clovaai/deep-text-recognition-benchmark)

In [28]:
%run ./../../../CRAFT-pytorch/test_modified.py \
    --trained_model="./../../../CRAFT-pytorch/craft_mlt_25k.pth" \
    --test_folder="./../../../data/thumbnailsHQ/Etap5"\
    --cuda=True \
    --result_folder="./../../../data/OCR_result/Etap5/"

./../../../data/OCR_result/Etap5/
Loading weights from checkpoint (./../../../CRAFT-pytorch/craft_mlt_25k.pth)
elapsed time : 1848.4327948093414sata/thumbnailsHQ/Etap5/__pjSbon3Kk.png


#### Wycięcie fragmentów obrazów z napisami

In [29]:
from skimage import io
import numpy as np
from pathlib import Path
import glob
import json

def genSlicesWithText(CRAFT_path_results=f"./../../../data/OCR_result/Etap5/", path_images=f"./../../../data/thumbnailsHQ/Etap5", out_df_path=f"./../../../our_data/Etap2/lilatee/Etap5-02_ocr.csv"):
    def bounding_box(points):
        x_coordinates, y_coordinates = zip(*points)
        return [(int(min(x_coordinates)), int(min(y_coordinates))), (int(max(x_coordinates)), int(max(y_coordinates)))]

    CRAFT_path_results = Path(CRAFT_path_results)
    path_images = Path(path_images)
    
    result_df = pd.DataFrame(columns=['video_id', 'word_id' ,'ocr_text', 'bbox'])
    for j, result_path in enumerate(glob.glob(str(CRAFT_path_results) + "/*.txt")):
        result_path = Path(result_path)
        video_id = result_path.stem[4:] # removing "res_" to obtain video_id
        
        img_path = Path(path_images / (video_id + ".png"))
        img = io.imread(img_path)
        shape = img.shape

        bboxes_path = CRAFT_path_results / ('res_' + img_path.stem + ".txt")
        bboxes = []
        with open(bboxes_path, "r") as bboxFile:
            for line in bboxFile.readlines():
                numbers = np.array(line.split(',')).astype(int)
                points = []

                for x,y in zip(numbers[::2], numbers[1::2]):
                    points.append((x, y))
                bboxes.append(bounding_box(points))


        if len(bboxes) > 0:
                Path(CRAFT_path_results / "words" / img_path.stem).mkdir(parents=True, exist_ok=True)
                img_dir = Path(CRAFT_path_results / "words" / img_path.stem)

        for i, bbox in enumerate(bboxes):
            img_slice = img[max(bbox[0][1], 0): min(bbox[1][1], shape[0]), max(bbox[0][0], 0):min(bbox[1][0], shape[1])]
            result_df = result_df.append({
                'video_id': video_id , 
                'word_id': i , 
                'ocr_text': '', 
                'bbox': json.dumps(bbox)}, ignore_index=True
            )
            
            io.imsave(img_dir / f"{i}.png", img_slice)
            
        if j % 100 == 0:
            print(j, end='\r')
            
    result_df.to_csv(out_df_path, index=False)

out_df_path = f"./../../../our_data/Etap5/Etap2/lilatee/02_ocr.csv"
genSlicesWithText(CRAFT_path_results=f"./../../../data/OCR_result/Etap5/", path_images=f"./../../../data/thumbnailsHQ/Etap5", out_df_path=out_df_path )

<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/-NEnDpcHkhY/2.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)
<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/-NEnDpcHkhY/6.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)
<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/-NEnDpcHkhY/8.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)
<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/-NEnDpcHkhY/10.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)
<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/-NEnDpcHkhY/11.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)
<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/-NEnDpcHkhY/14.png is a low contrast image
  io

<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/02ROLOZH-x4/0.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)


<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/0mdXIZzFRKE/0.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)


<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/3av1I7SoilY/0.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)


<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/3fqMyjd4F10/11.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)
<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/3fqMyjd4F10/15.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)


...

<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/TmjSVOrlHz4/1.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)


<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/VA2tHiM70uE/4.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)


<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/WK2WjnpdC84/5.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)


<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/wUBiK8fxkHY/1.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)


<ipython-input-29-72c51c74236b>:49: UserWarning: ../../../data/OCR_result/Etap5/words/ZuxAQIpKD30/5.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)


#### [Rozpoznawanie napisów - https://github.com/clovaai/CRAFT-pytorch](https://github.com/clovaai/CRAFT-pytorch)

In [ ]:
# If below does not work, comment "import utils" or "from utils import ..."

In [2]:
%run ./../../../deep-text-recognition-benchmark/demo_modified.py \
--Transformation TPS \
--FeatureExtraction ResNet \
--SequenceModeling BiLSTM \
--Prediction Attn \
--image_folder "./../../../data/OCR_result/Etap5/words/" \
--saved_model "./../../../deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth" \
--out_df_path "{out_df_path}"

/mnt/d/Documents/Uczelnia/Magisterka_Semestr3/PED/PED_YT/notebooks/Etap5/Etap2
model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from ./../../../deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth


### Łączenie wykrytych napisów w jeden, aby zapisać w kolumnie. Przed tym czyszczenie tzn.:
- usuwanie stopwords, 
- usuwanie słów krótszych niż 3, 
- lematyzacja,
- sprawdzenie czy słowo istnieje w słowniku - USUNIĘTO.

### CHECKPOINT

In [10]:
df_ocr = pd.read_csv(f"./../../../our_data/Etap5/Etap2/lilatee/02_ocr.csv")
df_ocr.loc[df_ocr.ocr_text.isna(),:] = ''
df_ocr[:5]

,video_id,word_id,ocr_text,bbox
0,--eizV2HaS0,0,wet,"[[88, 5], [130, 24]]"
1,--eizV2HaS0,1,8,"[[133, 5], [149, 24]]"
2,--eizV2HaS0,2,cool,"[[153, 5], [202, 24]]"
3,--eizV2HaS0,3,weather,"[[207, 5], [294, 23]]"
4,--eizV2HaS0,4,continues,"[[297, 4], [401, 26]]"


In [12]:
from utils import concatAndCleanOcrTextV2
df_ocr = df_ocr.groupby(by="video_id", axis=0).apply(lambda x: concatAndCleanOcrTextV2(x)).reset_index()
df_ocr.to_csv(f"./../../../our_data/Etap5/Etap2/lilatee/03_ocr_groupby.csv", index=False)
df_ocr[:10]

,video_id,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes
0,,,,[]
1,--SzddVKOnI,beme,beme,"[""[[21, 254], [164, 294]]""]"
2,--_jF4j66gA,ohnygodohmygon lit it got with voo shane dawso...,ohnygodohmygon lit get voo shane dawson work jhmv,"[""[[89, 191], [209, 207]]"", ""[[124, 212], [174..."
3,--eizV2HaS0,wet 8 cool weather continues now march 15 193 ...,wet cool weather continue march 193 2018 storm...,"[""[[69, 194], [88, 204]]"", ""[[286, 196], [308,..."
4,-0kVpJya5us,ireland ylog cravel,ireland ylog cravel,"[""[[20, 66], [316, 120]]"", ""[[215, 125], [280,..."
5,-1wRBPjLgp4,kevin joe keery pollakis keery goo chat show,kevin joe keery pollakis goo chat show,"[""[[60, 254], [122, 273]]"", ""[[153, 258], [182..."
6,-1zeOMJIEQY,pnu power 106 fm pusha 7 daytona release,pnu power 106 pusha daytona release,"[""[[356, 56], [410, 92]]"", ""[[13, 58], [85, 89..."
7,-4QhLG8xOsE,hd shemaroo ganga ki saugand,shemaroo ganga saugand,"[""[[401, 60], [465, 77]]"", ""[[4, 160], [144, 2..."
8,-5RCmu-HuTg,how to academy,academy,"[""[[24, 273], [77, 284]]""]"
9,-5tkXwu2I9A,hd,,[]


### Dodanie kolumny z wynikami nowego OCRa

In [16]:
df_ocr = df_ocr.set_index("video_id")
df = df.set_index("video_id").join(df_ocr).reset_index()
df["bboxes"] = df["bboxes"].fillna(json.dumps([]))
df.to_csv(f"./../../../our_data/Etap5/Etap2/lilatee/04_ocrV2.csv", index=False)
df[:5]

,video_id,thumbnail_link,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,mean_h,mean_s,mean_v,std_h,std_s,std_v,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes
0,tOpTQ3Fuf9s,https://i.ytimg.com/vi/tOpTQ3Fuf9s/default.jpg,89.439786,86.814786,91.361389,66.649746,64.668101,66.990815,"[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...",0.418176,0.139355,0.371523,0.377200,0.212690,0.270151,6.416138,\n,NaN,NaN,NaN,[]
1,k0DqRstCgj4,https://i.ytimg.com/vi/k0DqRstCgj4/default.jpg,44.614514,21.644572,18.150214,59.224952,44.159844,37.943564,"[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...",0.263084,0.458768,0.175262,0.410572,0.402156,0.232071,4.978147,Se!\n\n \n\nSMTOWN~\nOFFICIAL\n\nI\n,official,91 smtown official,smtown official,"[""[[109, 246], [189, 262]]"", ""[[108, 268], [19..."
2,1CYH4F1KCw8,https://i.ytimg.com/vi/1CYH4F1KCw8/default.jpg,89.025081,99.345885,116.328432,86.195281,90.707405,95.898890,"[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...",0.474761,0.326194,0.456745,0.252111,0.334823,0.375875,6.726162,\n,NaN,smtown official,smtown official,"[""[[109, 248], [189, 262]]"", ""[[108, 268], [19..."
3,UdjEoGzXxmI,https://i.ytimg.com/vi/UdjEoGzXxmI/default.jpg,128.491042,125.048414,122.029821,85.459664,82.416423,79.346087,"[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...","[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...","[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...",0.209823,0.111204,0.513423,0.275839,0.187658,0.333337,6.146261,\n,NaN,NaN,NaN,[]
4,cD8SYW8rjaQ,https://i.ytimg.com/vi/cD8SYW8rjaQ/default.jpg,57.089873,42.153490,36.812674,69.284572,58.369344,55.395476,"[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...","[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...","[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...",0.169770,0.377556,0.230208,0.240235,0.336576,0.273651,6.199159,\n,NaN,unions pagies smtown official,union pagies smtown official,"[""[[49, 94], [116, 113]]"", ""[[113, 94], [185, ..."


### Dodanie kolumny zawierającej średni punkt napisów w obrazie

In [17]:
def avg_texts_coords(bboxes):
    bboxes = json.loads(bboxes)
    if (len(bboxes) == 0):
        return json.dumps([-1, -1])
    middle_corrds_of_texts = []
    for bbox in bboxes:
        p1, p2 = json.loads(bbox)
        p1 = np.array(p1)
        p2 = np.array(p2)
        middle_text_coords = np.mean([p1, p2], axis=0)
        middle_corrds_of_texts.append(middle_text_coords)
#         print(p1, p2)
#         print(middle_text_coords)
    
    return json.dumps(list(np.mean(middle_corrds_of_texts, axis=0)))
    
df["avg_texts_coords"] = df.apply(lambda row: avg_texts_coords(row.bboxes), axis=1) 
df["avg_texts_coordX"] = df.apply(lambda row: json.loads(row.avg_texts_coords)[0],axis=1)
df["avg_texts_coordY"] = df.apply(lambda row: json.loads(row.avg_texts_coords)[1],axis=1)

df.to_csv(f"./../../../our_data/Etap5/Etap2/lilatee/05_middle_texts_point.csv", index=False)
df[:5]

,video_id,thumbnail_link,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,mean_h,mean_s,mean_v,std_h,std_s,std_v,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY
0,tOpTQ3Fuf9s,https://i.ytimg.com/vi/tOpTQ3Fuf9s/default.jpg,89.439786,86.814786,91.361389,66.649746,64.668101,66.990815,"[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...",0.418176,0.139355,0.371523,0.377200,0.212690,0.270151,6.416138,\n,NaN,NaN,NaN,[],"[-1, -1]",-1.00,-1.000
1,k0DqRstCgj4,https://i.ytimg.com/vi/k0DqRstCgj4/default.jpg,44.614514,21.644572,18.150214,59.224952,44.159844,37.943564,"[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...",0.263084,0.458768,0.175262,0.410572,0.402156,0.232071,4.978147,Se!\n\n \n\nSMTOWN~\nOFFICIAL\n\nI\n,official,91 smtown official,smtown official,"[""[[109, 246], [189, 262]]"", ""[[108, 268], [19...","[149.5, 265.25]",149.50,265.250
2,1CYH4F1KCw8,https://i.ytimg.com/vi/1CYH4F1KCw8/default.jpg,89.025081,99.345885,116.328432,86.195281,90.707405,95.898890,"[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...",0.474761,0.326194,0.456745,0.252111,0.334823,0.375875,6.726162,\n,NaN,smtown official,smtown official,"[""[[109, 248], [189, 262]]"", ""[[108, 268], [19...","[149.5, 265.75]",149.50,265.750
3,UdjEoGzXxmI,https://i.ytimg.com/vi/UdjEoGzXxmI/default.jpg,128.491042,125.048414,122.029821,85.459664,82.416423,79.346087,"[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...","[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...","[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...",0.209823,0.111204,0.513423,0.275839,0.187658,0.333337,6.146261,\n,NaN,NaN,NaN,[],"[-1, -1]",-1.00,-1.000
4,cD8SYW8rjaQ,https://i.ytimg.com/vi/cD8SYW8rjaQ/default.jpg,57.089873,42.153490,36.812674,69.284572,58.369344,55.395476,"[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...","[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...","[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...",0.169770,0.377556,0.230208,0.240235,0.336576,0.273651,6.199159,\n,NaN,unions pagies smtown official,union pagies smtown official,"[""[[49, 94], [116, 113]]"", ""[[113, 94], [185, ...","[132.75, 184.375]",132.75,184.375


### CHECKPOINT

In [18]:
df = pd.read_csv(f"./../../../our_data/Etap5/Etap2/lilatee/05_middle_texts_point.csv")
df[:5]

,video_id,thumbnail_link,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,mean_h,mean_s,mean_v,std_h,std_s,std_v,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY
0,tOpTQ3Fuf9s,https://i.ytimg.com/vi/tOpTQ3Fuf9s/default.jpg,89.439786,86.814786,91.361389,66.649746,64.668101,66.990815,"[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...",0.418176,0.139355,0.371523,0.377200,0.212690,0.270151,6.416138,\n,NaN,NaN,NaN,[],"[-1, -1]",-1.00,-1.000
1,k0DqRstCgj4,https://i.ytimg.com/vi/k0DqRstCgj4/default.jpg,44.614514,21.644572,18.150214,59.224952,44.159844,37.943564,"[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...",0.263084,0.458768,0.175262,0.410572,0.402156,0.232071,4.978147,Se!\n\n \n\nSMTOWN~\nOFFICIAL\n\nI\n,official,91 smtown official,smtown official,"[""[[109, 246], [189, 262]]"", ""[[108, 268], [19...","[149.5, 265.25]",149.50,265.250
2,1CYH4F1KCw8,https://i.ytimg.com/vi/1CYH4F1KCw8/default.jpg,89.025081,99.345885,116.328432,86.195281,90.707405,95.898890,"[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...",0.474761,0.326194,0.456745,0.252111,0.334823,0.375875,6.726162,\n,NaN,smtown official,smtown official,"[""[[109, 248], [189, 262]]"", ""[[108, 268], [19...","[149.5, 265.75]",149.50,265.750
3,UdjEoGzXxmI,https://i.ytimg.com/vi/UdjEoGzXxmI/default.jpg,128.491042,125.048414,122.029821,85.459664,82.416423,79.346087,"[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...","[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...","[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...",0.209823,0.111204,0.513423,0.275839,0.187658,0.333337,6.146261,\n,NaN,NaN,NaN,[],"[-1, -1]",-1.00,-1.000
4,cD8SYW8rjaQ,https://i.ytimg.com/vi/cD8SYW8rjaQ/default.jpg,57.089873,42.153490,36.812674,69.284572,58.369344,55.395476,"[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...","[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...","[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...",0.169770,0.377556,0.230208,0.240235,0.336576,0.273651,6.199159,\n,NaN,unions pagies smtown official,union pagies smtown official,"[""[[49, 94], [116, 113]]"", ""[[113, 94], [185, ...","[132.75, 184.375]",132.75,184.375


### Analiza sentymentu

In [21]:
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

df = pd.read_csv(f"./../../../our_data/Etap5/Etap2/lilatee/05_middle_texts_point.csv")

sia = SentimentIntensityAnalyzer()

df["clean_thumb_ocr_text_V2"] = df["clean_thumb_ocr_text_V2"].fillna('')
df["ocr_sentiment_compound"] = df.apply(lambda row: sia.polarity_scores(row.clean_thumb_ocr_text_V2)['compound'] ,axis=1)
df.to_csv(f"./../../../our_data/Etap5/Etap2/lilatee/06_sentiment.csv", index=False)
df[:10]

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/lilatee/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,video_id,thumbnail_link,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,mean_h,mean_s,mean_v,std_h,std_s,std_v,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,ocr_sentiment_compound
0,tOpTQ3Fuf9s,https://i.ytimg.com/vi/tOpTQ3Fuf9s/default.jpg,89.439786,86.814786,91.361389,66.649746,64.668101,66.990815,"[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...",0.418176,0.139355,0.371523,0.377200,0.212690,0.270151,6.416138,\n,NaN,NaN,,[],"[-1, -1]",-1.000,-1.0000,0.0000
1,k0DqRstCgj4,https://i.ytimg.com/vi/k0DqRstCgj4/default.jpg,44.614514,21.644572,18.150214,59.224952,44.159844,37.943564,"[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...",0.263084,0.458768,0.175262,0.410572,0.402156,0.232071,4.978147,Se!\n\n \n\nSMTOWN~\nOFFICIAL\n\nI\n,official,91 smtown official,smtown official,"[""[[109, 246], [189, 262]]"", ""[[108, 268], [19...","[149.5, 265.25]",149.500,265.2500,0.0000
2,1CYH4F1KCw8,https://i.ytimg.com/vi/1CYH4F1KCw8/default.jpg,89.025081,99.345885,116.328432,86.195281,90.707405,95.898890,"[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...",0.474761,0.326194,0.456745,0.252111,0.334823,0.375875,6.726162,\n,NaN,smtown official,smtown official,"[""[[109, 248], [189, 262]]"", ""[[108, 268], [19...","[149.5, 265.75]",149.500,265.7500,0.0000
3,UdjEoGzXxmI,https://i.ytimg.com/vi/UdjEoGzXxmI/default.jpg,128.491042,125.048414,122.029821,85.459664,82.416423,79.346087,"[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...","[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...","[38336, 2281, 961, 535, 529, 342, 71, 137, 7, ...",0.209823,0.111204,0.513423,0.275839,0.187658,0.333337,6.146261,\n,NaN,NaN,,[],"[-1, -1]",-1.000,-1.0000,0.0000
4,cD8SYW8rjaQ,https://i.ytimg.com/vi/cD8SYW8rjaQ/default.jpg,57.089873,42.153490,36.812674,69.284572,58.369344,55.395476,"[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...","[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...","[39520, 3789, 3258, 4917, 3411, 2838, 2066, 19...",0.169770,0.377556,0.230208,0.240235,0.336576,0.273651,6.199159,\n,NaN,unions pagies smtown official,union pagies smtown official,"[""[[49, 94], [116, 113]]"", ""[[113, 94], [185, ...","[132.75, 184.375]",132.750,184.3750,0.0000
5,8b2EP0NZtrU,https://i.ytimg.com/vi/8b2EP0NZtrU/default.jpg,105.325365,103.370110,106.009763,69.802237,68.667124,68.402935,"[38760, 2621, 1618, 664, 729, 401, 398, 428, 3...","[38760, 2621, 1618, 664, 729, 401, 398, 428, 3...","[38760, 2621, 1618, 664, 729, 401, 398, 428, 3...",0.329132,0.170262,0.446202,0.310222,0.235154,0.278957,6.085746,\n\n \n,NaN,NaN,,[],"[-1, -1]",-1.000,-1.0000,0.0000
6,X-L40JAuaGU,https://i.ytimg.com/vi/X-L40JAuaGU/default.jpg,37.845098,83.754444,120.585955,69.988590,77.404157,85.363350,"[42255, 2364, 1386, 1818, 1597, 1892, 2348, 24...","[42255, 2364, 1386, 1818, 1597, 1892, 2348, 24...","[42255, 2364, 1386, 1818, 1597, 1892, 2348, 24...",0.448399,0.605108,0.473836,0.249338,0.401595,0.335614,6.644959,ORIGINAL BROADWAY CAST RECORDING\nKRISTEN ANDE...,original broadway cast record pea,recording original broadway cast frozen the br...,record original broadway cast frozen musical m...,"[""[[380, 64], [462, 80]]"", ""[[184, 65], [253, ...","[340.625, 189.5625]",340.625,189.5625,0.3182
7,1wRW8eh6hGI,https://i.ytimg.com/vi/1wRW8eh6hGI/default.jpg,83.362980,83.837969,94.498356,81.635012,78.642142,78.567202,"[40670, 1482, 375, 1269, 852, 432, 697, 497, 4...","[40670, 1482, 375, 1269, 852, 432, 697, 497, 4...","[40670, 1482, 375, 1269, 852, 432, 697, 497, 4...",0.475297,0.461910,0.463048,0.361660,0.362075,0.337380,6.708425,\n,NaN,vevo,vevo,"[""[[12, 277], [101, 305]]""]","[56.5, 291.0]",56.

### HSV

In [22]:
from skimage import io
from skimage.color import rgb2hsv
from pathlib import Path
import glob

df = pd.read_csv(f"./../../../our_data/Etap5/Etap2/lilatee/06_sentiment.csv")
imgs_path = Path(f'./../../../data/thumbnailsHQ/Etap5')

for i, img_path in enumerate(glob.glob(f"{imgs_path}/*.png")): 
    path = Path(img_path)
    img = io.imread(path)
    img = rgb2hsv(img)

    list_of_pixels = img.reshape(img.shape[0]*img.shape[1], 3)
    mean_of_channels = np.mean(list_of_pixels, axis=0)
    std_of_channels = np.std(list_of_pixels, axis=0)

    df.loc[df.video_id == path.stem, "mean_h"] = mean_of_channels[0]
    df.loc[df.video_id == path.stem, "mean_s"] = mean_of_channels[1]
    df.loc[df.video_id == path.stem, "mean_v"] = mean_of_channels[2]
    
    df.loc[df.video_id == path.stem, "std_h"] = std_of_channels[0]
    df.loc[df.video_id == path.stem, "std_s"] = std_of_channels[1]
    df.loc[df.video_id == path.stem, "std_v"] = std_of_channels[2]
    if (i % 100 == 0):
        print(i, end='\r')

df.to_csv(f"./../../../our_data/Etap5/Etap2/lilatee/06_sentiment_hsv.csv", index=False)

### Utworzenie wektora cech z ResNet18

In [51]:
# %%time
# import torch
# import torchvision
# import torch.nn as nn
# import torchvision.models as models
# import torchvision.transforms as transforms
# from torch.autograd import Variable
# from PIL import Image

# # region = 'US'
# df = pd.read_csv(f"./../../../our_data/Etap5/Etap2/lilatee/06_sentiment_hsv.csv")

# # Odkomentować zapis jeśli trzeba!!!
# imgs_path = Path(f'./../../../data/thumbnailsHQ/{region}')

# # # Load the pretrained model
# model = models.resnet18(pretrained=True)
# # model = torch.hub.load('pytorch/vision:v0.9.0', 'resnet18', pretrained=True)
# layer = model._modules.get('avgpool')
# model.eval()

# preprocess = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])


# for i, img_path in enumerate(glob.glob(f"{imgs_path}/*.png")): 
#     path = Path(img_path)
#     imgPIL = Image.open(path)
    
#     input_tensor = preprocess(imgPIL)
#     input_batch = input_tensor.unsqueeze(0)
    
#     # move the input and model to GPU for speed if available
#     if torch.cuda.is_available():
#         input_batch = input_batch.to('cuda')
#         model.to('cuda')
    
#     features_vector = torch.zeros(512)
#     def copy_data(module, input_, output):
#         features_vector.copy_(output.data.flatten())
        
#     h = layer.register_forward_hook(copy_data)
    
#     with torch.no_grad():
#         output = model(input_batch)

#     df.loc[df.video_id == path.stem, "features_vec_resnet18"] = json.dumps(features_vector.tolist())    

#     if (i % 100 == 0):
#         print(i, end='\r')


# df.to_csv(f"./../../../our_data/Etap5/Etap2/lilatee/07_ResNet.csv", index=False)
# df[:5]

CPU times: user 16min 59s, sys: 2min 4s, total: 19min 4s
Wall time: 19min 18s


,video_id,thumbnail_link,region,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,features_vec_resnet18
0,2kyS6SvSYSE,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,US,83.791198,77.128183,74.980914,78.653796,73.570512,73.087883,"[38336, 2426, 1188, 690, 848, 602, 597, 711, 1...","[38336, 2426, 1188, 690, 848, 602, 597, 711, 1...","[38336, 2426, 1188, 690, 848, 602, 597, 711, 1...",6.818234,\n,NaN,NaN,NaN,[],"[-1, -1]",-1.00,-1.000000,"[0.028933335095643997, 1.5294386148452759, 0.4..."
1,1ZAPwfrtAFY,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,US,53.131123,50.240133,55.462459,66.342402,62.884021,62.213755,"[48274, 4068, 2082, 1566, 1196, 1059, 1116, 10...","[48274, 4068, 2082, 1566, 1196, 1059, 1116, 10...","[48274, 4068, 2082, 1566, 1196, 1059, 1116, 10...",6.185228,\n,NaN,the trump presidency,trump presidency,"[""[[68, 169], [105, 178]]"", ""[[108, 169], [176...","[114.25, 173.5]",114.25,173.500000,"[0.34424084424972534, 2.8481504917144775, 0.59..."
2,5qpjK5DgCt4,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],"[-1, -1]",-1.00,-1.000000,NaN
3,puqaWrEC7tY,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,US,85.942714,86.987807,84.269149,85.556903,83.919807,85.287368,"[46590, 3405, 1702, 1125, 946, 831, 860, 821, ...","[46590, 3405, 1702, 1125, 946, 831, 860, 821, ...","[46590, 3405, 1702, 1125, 946, 831, 860, 821, ...",6.434697,\n,NaN,this at look photograph gmm youlube originals,look photograph original,"[""[[262, 169], [334, 216]]"", ""[[269, 190], [41...","[289.0, 235.66666666666666]",289.00,235.666667,"[0.10734694451093674, 3.141456365585327, 1.885..."
4,d380meD0W0M,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,US,111.840347,103.171238,102.399716,104.868991,96.903546,93.691704,"[47085, 4968, 3326, 2840, 2207, 1674, 1422, 12...","[47085, 4968, 3326, 2840, 2207, 1674, 1422, 12...","[47085, 4968, 3326, 2840, 2207, 1674, 1422, 12...",6.510005,\n,NaN,going baldzi dare youi,dare,"[""[[0, 262], [94, 297]]""]","[47.0, 279.5]",47.00,279.500000,"[0.8179275393486023, 2.0939841270446777, 0.666..."


### Połączenie wyników z Etapem 1

In [23]:
import pandas as pd
df_1 = pd.read_csv(f"./../../../our_data/Etap5/Etap1/checkpoint1.csv")
df_2 = pd.read_csv(f"./../../../our_data/Etap5/Etap2/lilatee/06_sentiment_hsv.csv")
print(df_1.shape)
print(df_2.shape)

/home/lilatee/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(8474, 69)
(8474, 27)


In [24]:
df_1[:3]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,relatedTo,pub_date,pub_day_of_the_week,pub_year,pub_month,pub_day,pub_hour,tren_date,tren_day_of_the_week,tren_year,tren_month,tren_day,time_to_trend_in_days,dislikes/likes,likes/views,dislikes/views,comment_count/views,increase_views,increase_likes,increase_dislikes,increase_comms,times_in_trend,avg_views,avg_likes,avg_dislikes,avg_comms,avg_views_increase_per_hour,n_of_tags,title_clean,title_length,title_n_of_words,title_capital_letters,title_capital_letters_percent,title_small_letters,title_small_letters_percent,title_punctuation,title_punctuation_percent,title_n_of_emojis,desc_clean,desc_length,desc_n_of_words,desc_capital_letters,desc_capital_letters_percent,desc_small_letters,desc_small_letters_percent,desc_punctuation,desc_punctuation_percent,desc_list_of_urls,desc_n_of_urls,desc_n_of_emojis,isFacebook,isTwitter,isInstagram
0,tOpTQ3Fuf9s,NaN,NCT 2018 엔시티 2018 &#39;Black on Black&#39; Dan...,SMTOWN,10,2018-04-22 12:00:03,"[""NCT"", ""NCT 2018"", ""TAEYONG"", ""JAEHYUN"", ""JUN...",23641741.0,687477.0,2758.0,19889.0,https://i.ytimg.com/vi/tOpTQ3Fuf9s/default.jpg,NaN,NaN,NaN,NCT 2018 엔시티 2018 'Black on Black' Dance Pract...,NaN,2018-04-22 12:00:03,6.0,2018.0,4.0,22.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,0.004012,0.029079,0.000117,0.000841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,nct 2018 엔시티 2018 39black black39 dance practice,57,9,7,0.122807,21,0.368421,6,0.105263,0,nct 2018 엔시티 2018 black black dance practice ℗...,69,12,10,0.144928,33,0.478261,3,0.043478,[],0,0,False,False,False
1,k0DqRstCgj4,NaN,NCT U 엔시티 유 &#39;Baby Don&#39;t Stop&#39; MV,SMTOWN,10,2018-02-26 15:00:05,"[""NCT"", ""NCT 2018"", ""TAEYONG"", ""JAEHYUN"", ""JUN...",83681971.0,1815611.0,10975.0,85082.0,https://i.ytimg.com/vi/k0DqRstCgj4/default.jpg,NaN,NaN,NaN,[Tracklist] 01 INTRO: Neo Got My Back 02 BOSS ...,NaN,2018-02-26 15:00:05,0.0,2018.0,2.0,26.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,0.006045,0.021697,0.000131,0.001017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54,nct u 엔시티 유 39baby don39t stop39 mv,44,8,9,0.204545,9,0.204545,9,0.204545,0,tracklist 01 intro neo get back 02 bos 03 baby...,140,30,35,0.250000,42,0.300000,9,0.064286,[],0,0,False,False,False
2,1CYH4F1KCw8,NaN,NCT 2018 엔시티 2018 &#39;Black on Black&#39; MV ...,SMTOWN,10,2018-04-18 15:00:01,"[""NCT"", ""NCT 2018"", ""TAEYONG"", ""JAEHYUN"", ""JUN...",50414057.0,1462369.0,10411.0,68820.0,https://i.ytimg.com/vi/1CYH4F1KCw8/default.jpg,NaN,NaN,NaN,[Tracklist] 01 INTRO: Neo Got My Back 02 BOSS ...,NaN,2018-04-18 15:00:01,2.0,2018.0,4.0,18.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,0.007119,0.029007,0.000207,0.001365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,nct 2018 엔시티 2018 39black black39 mv performan...,64,10,9,0.140625,22,0.343750,9,0.140625,0,tracklist 01 intro neo get back 02 bos 03 baby...,140,30,35,0.250000,42,0.300000,9,0.064286,[],0,0,False,False,False


In [25]:
df_2[:3]

,video_id,thumbnail_link,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,mean_h,mean_s,mean_v,std_h,std_s,std_v,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,ocr_sentiment_compound
0,tOpTQ3Fuf9s,https://i.ytimg.com/vi/tOpTQ3Fuf9s/default.jpg,89.439786,86.814786,91.361389,66.649746,64.668101,66.990815,"[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...",0.418176,0.139355,0.371523,0.377200,0.212690,0.270151,6.416138,\n,NaN,NaN,NaN,[],"[-1, -1]",-1.0,-1.00,0.0
1,k0DqRstCgj4,https://i.ytimg.com/vi/k0DqRstCgj4/default.jpg,44.614514,21.644572,18.150214,59.224952,44.159844,37.943564,"[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...",0.263084,0.458768,0.175262,0.410572,0.402156,0.232071,4.978147,Se!\n\n \n\nSMTOWN~\nOFFICIAL\n\nI\n,official,91 smtown official,smtown official,"[""[[109, 246], [189, 262]]"", ""[[108, 268], [19...","[149.5, 265.25]",149.5,265.25,0.0
2,1CYH4F1KCw8,https://i.ytimg.com/vi/1CYH4F1KCw8/default.jpg,89.025081,99.345885,116.328432,86.195281,90.707405,95.898890,"[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...","[45247, 5867, 3033, 2296, 1826, 1635, 1642, 14...",0.474761,0.326194,0.456745,0.252111,0.334823,0.375875,6.726162,\n,NaN,smtown official,smtown official,"[""[[109, 248], [189, 262]]"", ""[[108, 268], [19...","[149.5, 265.75]",149.5,265.75,0.0


In [26]:
df_result = pd.concat([df_1.set_index(df_1.video_id), df_2.set_index(df_2.video_id).iloc[:, 2:]], axis=1)
df_result.to_csv(f"./../../../our_data/Etap5/Etap2/Etap2LiLateeFinal.csv", index=False)
df_result[:3]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,relatedTo,pub_date,pub_day_of_the_week,pub_year,pub_month,pub_day,pub_hour,tren_date,tren_day_of_the_week,tren_year,tren_month,tren_day,time_to_trend_in_days,dislikes/likes,likes/views,dislikes/views,comment_count/views,increase_views,increase_likes,increase_dislikes,increase_comms,times_in_trend,avg_views,avg_likes,avg_dislikes,avg_comms,avg_views_increase_per_hour,n_of_tags,title_clean,title_length,title_n_of_words,title_capital_letters,title_capital_letters_percent,title_small_letters,title_small_letters_percent,title_punctuation,title_punctuation_percent,title_n_of_emojis,desc_clean,desc_length,desc_n_of_words,desc_capital_letters,desc_capital_letters_percent,desc_small_letters,desc_small_letters_percent,desc_punctuation,desc_punctuation_percent,desc_list_of_urls,desc_n_of_urls,desc_n_of_emojis,isFacebook,isTwitter,isInstagram,mean_r,mean_g,mean_b,std_r,std_g,std_b,hist_r,hist_g,hist_b,mean_h,mean_s,mean_v,std_h,std_s,std_v,entropy,thumb_ocr_text,clean_thumb_ocr_text,thumb_ocr_text_V2,clean_thumb_ocr_text_V2,bboxes,avg_texts_coords,avg_texts_coordX,avg_texts_coordY,ocr_sentiment_compound
video_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
tOpTQ3Fuf9s,tOpTQ3Fuf9s,NaN,NCT 2018 엔시티 2018 &#39;Black on Black&#39; Dan...,SMTOWN,10,2018-04-22 12:00:03,"[""NCT"", ""NCT 2018"", ""TAEYONG"", ""JAEHYUN"", ""JUN...",23641741.0,687477.0,2758.0,19889.0,https://i.ytimg.com/vi/tOpTQ3Fuf9s/default.jpg,NaN,NaN,NaN,NCT 2018 엔시티 2018 'Black on Black' Dance Pract...,NaN,2018-04-22 12:00:03,6.0,2018.0,4.0,22.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,0.004012,0.029079,0.000117,0.000841,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,nct 2018 엔시티 2018 39black black39 dance practice,57,9,7,0.122807,21,0.368421,6,0.105263,0,nct 2018 엔시티 2018 black black dance practice ℗...,69,12,10,0.144928,33,0.478261,3,0.043478,[],0,0,False,False,False,89.439786,86.814786,91.361389,66.649746,64.668101,66.990815,"[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...","[38701, 2870, 986, 721, 672, 703, 636, 663, 79...",0.418176,0.139355,0.371523,0.377200,0.212690,0.270151,6.416138,\n,NaN,NaN,NaN,[],"[-1, -1]",-1.0,-1.00,0.0
k0DqRstCgj4,k0DqRstCgj4,NaN,NCT U 엔시티 유 &#39;Baby Don&#39;t Stop&#39; MV,SMTOWN,10,2018-02-26 15:00:05,"[""NCT"", ""NCT 2018"", ""TAEYONG"", ""JAEHYUN"", ""JUN...",83681971.0,1815611.0,10975.0,85082.0,https://i.ytimg.com/vi/k0DqRstCgj4/default.jpg,NaN,NaN,NaN,[Tracklist] 01 INTRO: Neo Got My Back 02 BOSS ...,NaN,2018-02-26 15:00:05,0.0,2018.0,2.0,26.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,0.006045,0.021697,0.000131,0.001017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54,nct u 엔시티 유 39baby don39t stop39 mv,44,8,9,0.204545,9,0.204545,9,0.204545,0,tracklist 01 intro neo get back 02 bos 03 baby...,140,30,35,0.250000,42,0.300000,9,0.064286,[],0,0,False,False,False,44.614514,21.644572,18.150214,59.224952,44.159844,37.943564,"[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...","[67937, 2020, 1977, 1533, 1487, 1517, 1441, 12...",0.263084,0.458768,0.175262,0.410572,0.402156,0.232071,4.978147,Se!\n\n \n\nSMTOWN~\nOFFICIAL\n\nI\n,official,91 smtown official,smtown official,"[""[[109, 246], [189, 262]]"", ""[[108, 268], [19...","[149.5, 265.25]",149.5,265.25,0.0
1CYH4F1KCw8,1CYH4F1KCw8,NaN,NCT 2018 엔시티 2018 &#39;Black on Black&#39; MV ...,SMTOWN,10,2018-04-18 15:00:01,"[""NCT"", ""NCT 2018"", ""TAEYONG"", ""JAEHYUN"", ""JUN...",50414057.0,1462369.0,10411.0,68820.0,https://i.ytimg.com/vi/1CYH4F1KCw8/default.jpg,NaN,NaN,NaN,[Tracklist] 01 INTRO: Neo Got My Back 02 BOSS ...,NaN,2018-04-18 15:00:01,2.0,2018.0,4.0,18.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,0.007119,0.029007,0.000207,0.001365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,nct 2018 엔시티 2018 39black bl